In [ ]:
import numpy as np
import pandas as pd
import random
import time
import requests
import re
import os
from io import StringIO

# Import column dictionary
from column_dictionary import (
    TABLE_CONFIGS,
    get_column_mapping,
    get_headers,
    needs_header_fix,
)

# Output directory
OUTPUT_DIR = "temp_csv"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# NFL team abbreviations (Pro-Football-Reference format)
nfl_teams = [
    "crd", "atl", "rav", "buf", "car", "chi", "cin", "cle",
    "dal", "den", "det", "gnb", "htx", "clt", "jax", "kan",
    "rai", "sdg", "ram", "mia", "min", "nwe", "nor", "nyg",
    "nyj", "phi", "pit", "sfo", "sea", "tam", "oti", "was"
]

# URLs
team_rankings_url = "https://www.pro-football-reference.com/years/2025/"
defensive_rankings_url = "https://www.pro-football-reference.com/years/2025/opp.htm"

def get_team_profile_url(team_abbr):
    return f"https://www.pro-football-reference.com/teams/{team_abbr}/2025.htm"


def clean_dataframe(df, table_type):
    """
    Clean a dataframe by fixing headers and renaming columns.
    
    Args:
        df: The dataframe to clean
        table_type: The type of table (e.g., 'passing_offense', 'schedule')
    
    Returns:
        Cleaned dataframe
    """
    # Get column mapping
    column_mapping = get_column_mapping(table_type)
    
    # Rename columns that exist in the mapping
    rename_dict = {old: new for old, new in column_mapping.items() if old in df.columns}
    if rename_dict:
        df = df.rename(columns=rename_dict)
    
    return df


def extract_tables(url, tables, output_dir=OUTPUT_DIR, prefix=""):
    """
    Extract tables from any Pro-Football-Reference URL.
    Automatically handles tables hidden in HTML comments.
    Fixes headers and renames columns using column_dictionary.
    Skips files that already exist.
    
    Args:
        url: The page URL to scrape
        tables: Dict mapping table_id -> (filename, table_type)
        output_dir: Directory to save CSV files
        prefix: Optional prefix for filenames (e.g., team abbr)
    
    Returns:
        Dict of successfully extracted DataFrames
    """
    results = {}
    tables_to_fetch = {}
    
    # Check which files already exist
    for table_id, (filename, table_type) in tables.items():
        full_filename = f"{prefix}{filename}" if prefix else filename
        filepath = f"{output_dir}/{full_filename}.csv"
        if os.path.exists(filepath):
            print(f"⏭ Skipping {full_filename}.csv (already exists)")
        else:
            tables_to_fetch[table_id] = (full_filename, table_type)
    
    # If all files exist, skip the HTTP request
    if not tables_to_fetch:
        return results
    
    # Fetch the page
    response = requests.get(url)
    html = response.text
    
    # Extract HTML comments (where PFR hides some tables)
    comments = re.findall(r'<!--(.+?)-->', html, re.DOTALL)
    
    for table_id, (filename, table_type) in tables_to_fetch.items():
        df = None
        
        # Determine header row based on table type
        header_row = 0 if needs_header_fix(table_type) else 1
        
        # Try 1: Direct extraction from visible HTML
        try:
            df = pd.read_html(StringIO(html), header=header_row, attrs={'id': table_id})[0]
        except (ValueError, IndexError):
            pass
        
        # Try 2: Extract from HTML comments
        if df is None:
            for comment in comments:
                if f'id="{table_id}"' in comment:
                    try:
                        df = pd.read_html(StringIO(comment), header=header_row, attrs={'id': table_id})[0]
                        break
                    except (ValueError, IndexError):
                        continue
        
        # Process and save if found
        if df is not None:
            # Clean the dataframe (rename columns)
            df = clean_dataframe(df, table_type)
            
            # Save to CSV
            filepath = f"{output_dir}/{filename}.csv"
            df.to_csv(filepath, index=False)
            results[filename] = df
            print(f"✓ Saved {filename}.csv ({len(df)} rows, {len(df.columns)} cols)")
        else:
            print(f"✗ Table '{table_id}' not found")
    
    return results


# =============================================================================
# 1. TEAM RANKINGS (Offensive)
# =============================================================================
# Format: table_id -> (filename, table_type)
ranking_tables = {
    "team_stats": ("team_offense", "team_offense"),
    "passing": ("passing_offense", "passing_offense"),
    "rushing": ("rushing_offense", "rushing_offense"),
    "team_scoring": ("scoring_offense", "scoring_offense"),
    "AFC": ("afc_standings", "afc_standings"),
    "NFC": ("nfc_standings", "nfc_standings"),
}

print("=" * 50)
print("FETCHING TEAM RANKINGS (OFFENSE)")
print("=" * 50)
results = extract_tables(team_rankings_url, ranking_tables)
print(f"Extracted {len(results)} tables.\n")

# =============================================================================
# 2. DEFENSIVE RANKINGS
# =============================================================================
defensive_tables = {
    "team_stats": ("team_defense", "team_defense"),
    "advanced_defense": ("advanced_defense", "advanced_defense"),
    "passing": ("passing_defense", "passing_defense"),
    "rushing": ("rushing_defense", "rushing_defense"),
}

print("=" * 50)
print("FETCHING DEFENSIVE RANKINGS")
print("=" * 50)
results = extract_tables(defensive_rankings_url, defensive_tables)
print(f"Extracted {len(results)} tables.\n")

# =============================================================================
# 3. TEAM PROFILES (32 teams)
# =============================================================================
# Format: table_id -> (filename, table_type)
profile_tables = {
    "team_stats": ("team_stats", "team_stats"),
    "games": ("schedule", "schedule"),
    "passing": ("passing", "passing"),
    "rushing_and_receiving": ("rushing_receiving", "rushing_receiving"),
    "defense": ("defense", "defense"),
    "scoring": ("scoring", "scoring"),
    "team_td_log": ("touchdown_log", "touchdown_log"),
}

def get_profile_tables_for_team(team_abbr):
    tables = profile_tables.copy()
    tables[f"{team_abbr}_injury_report"] = ("injury_report", "injury_report")
    return tables

print("=" * 50)
print("FETCHING TEAM PROFILES (32 teams)")
print("=" * 50)

for i, team in enumerate(nfl_teams):
    print(f"\n[{i+1}/32] {team.upper()}")
    print("-" * 30)
    
    url = get_team_profile_url(team)
    tables = get_profile_tables_for_team(team)
    
    results = extract_tables(url, tables, prefix=f"{team}_")
    
    # Rate limiting - 3 seconds between teams
    if i < len(nfl_teams) - 1:
        time.sleep(3)

print("\n" + "=" * 50)
print("DONE!")
print("=" * 50)